Please use the Local ./Dockerfile environment 

In [1]:
# install the faiss-cpu when new env is provisioned
#!pip install faiss-cpu
#conda install -c pytorch faiss-cpu=1.7.4 mkl=2021

In [2]:
# install the openai when new env is provisioned
#!pip install openai

#conda install -c conda-forge openai

In [3]:
import os
import umap
import faiss
import openai
import pandas as pd
import numpy as np
import altair as alt
import configparser
#from sklearn import decomposition

import warnings
warnings.filterwarnings('ignore')

alt.data_transformers.disable_max_rows()


c:\Users\hoge\Miniconda3\envs\py37\envs\py39openai\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\hoge\Miniconda3\envs\py37\envs\py39openai\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\hoge\Miniconda3\envs\py37\envs\py39openai\lib\site-

DataTransformerRegistry.enable('default')

In [4]:
pd.__version__

'1.5.3'

In [5]:
# make sure we are using FAISS version 1.7.4 
faiss.__version__

'1.7.4'

### Prepare required resources

In [6]:
# This dataframe is created with the amazon_reviews_us_Apparel_v1_00.tsv.gz dataset with following filtering criteria:
# 1. Year 2015
# 2. Products with 15 - 25 reviews
# 3. review_lenght > 10

df_apparel = pd.read_csv('../resources/data/apparel_10to14.tsv.gz', sep='\t', compression='gzip')
cols = ['product_id', 'review_id', 'star_rating', 'product_title', 'review_body']
df_apparel = df_apparel[cols]
df_apparel.head(3)

,product_id,review_id,star_rating,product_title,review_body
0,B00001QHXX,R2HBUQ97RV5JVR,3.0,Richard Nixon Mask,I got this mask for a company party and it fre...
1,B00001QHXX,RHCH92YNAS282,4.0,Richard Nixon Mask,"Nice mask, will have to do some fitting work. ..."
2,B00001QHXX,R1OHYB07D0WE35,5.0,Richard Nixon Mask,"Even though its a bit large, I can't help but ..."


In [7]:
# Load the pre-created FAISS index
faiss_index = faiss.read_index('../resources/binary/apparel_10to14_review_cosine.faissindex')

In [8]:
# Load your API key from an environment variable or secret management service
config = configparser.ConfigParser()
config.read('nes.ini')
openai.api_key = config['OpenAI']['api_key']

### Define helper functions

In [9]:
# Helper function to create the query embedding. Make sure to use the same model as what we used to created the product embedding
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   
   return np.array(openai.Embedding.create(input = [text], model=model)['data'][0]['embedding'], dtype='float32').reshape(1, -1)

In [10]:
def search_with_original_query(df, faiss_index, query_embedding, num_of_records=100):
    # we need to normalize the question embedding in order to use cosine similarity to search 
    faiss.normalize_L2(query_embedding)

    # distance is the correspnding distance
    # result_idx is the index of the input array, hence the index of the dataframe (if the dataframe index is reset which starts with 0)
    distance, result_idx = faiss_index.search(query_embedding, k=num_of_records)

    # use the return index to create the result dataframe
    df_result = df.iloc[result_idx.squeeze().tolist()]
    # add Distance to the result dataframe
    df_result['distance'] = distance.T

    df_result = df_result.sort_values(by='distance', ascending=True)
    
    return df_result, result_idx

In [11]:
prompt_rewrite ="""
    You are an English teacher. You need to find every single adjective from the sentences delimited by triple backquotes below.
    Then you rewrite the sentences by changing the adjectives in their opposite meaning. 
    Finally, you only need to output the rewritten sentences.

    ```{}```
"""

prompt_antonym="""
    You are an English teacher. You need to find every single ADJECTIVE from the sentences delimited by triple backquotes below.
    Then, you transform every adjective into its antonym.
    Finally, give the dictionary meaning for each antonym.
    Below are two examples. You need to comlete the third one. 
    

    Text 1: Kids flip flops for girl, cute, good fit, comfortable and durable, low price
    Output 1: Artless means without guile or deception. Unsuited means not proper or fitting for something. Uncomfortable means causing discomfort.  Fragile means easily broken. Costly means expensive.
    ## 
    Text 2: Long sleeve shirts for men. Wrinkle-free, thick but breathable and slim fit
    Output 2: Short means having little length. Crinkle means to form many short bends or ripples. Thin means measuring little in cross section or diameter. Airtight means impermeable to air or nearly so. Wide means having a greater than usual measure across
    ##
    Text 3:  ```{}```
    Output 3:
"""

In [12]:
def generate_opposite_query(orignal_query='', prompt=''):

    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prompt.format(orignal_query),
        temperature=0,
        max_tokens=1000,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )

    return response['choices'][0]['text']

In [13]:
def search_with_opposite_query(df, faiss_index, opposite_query_embedding, original_query_result_index, num_of_records=100):

    faiss.normalize_L2(opposite_query_embedding)

    # we want to make sure the opposite query only compare against the texts found by the original query 
    id_selector = faiss.IDSelectorArray(original_query_result_index.shape[1], faiss.swig_ptr(original_query_result_index))
    filtered_distances, filtered_indices = faiss_index.search(opposite_query_embedding, k=num_of_records, params=faiss.SearchParametersIVF(sel=id_selector))

    df_opposite_result = df.iloc[filtered_indices.squeeze().tolist()]
    df_opposite_result['distance'] = filtered_distances.T

    df_opposite_result = df_opposite_result.sort_values(by='distance', ascending=False)

    return df_opposite_result

In [14]:
def get_reconcile_result(df_result_original, df_result_opposite):

    df_reconcile_result = df_result_original.merge(df_result_opposite[['review_id', 'distance']], 
                            left_on='review_id', right_on='review_id', how='left', suffixes=('_original', '_opposite'))
    
    # Using Dot Product FAISS Index with L2 normaliztion, the returning result is Cosine Similiarty, rather than Distance.
    # There will turn the Cosine Similarity to Distance 
    df_reconcile_result['distance_original'] = 1 - df_reconcile_result['distance_original']
    df_reconcile_result['distance_opposite'] = 1 - df_reconcile_result['distance_opposite']

    df_reconcile_result = df_reconcile_result.sort_values(by='distance_original', ascending=True).reset_index(drop=True)

    return df_reconcile_result

In [15]:
# Helper function to clip the distance_opposite

def clip_distance_opposite(df, clipping=0.5):
    df = df.sort_values(by='distance_opposite', ascending=False).reset_index(drop=True)
    # Flatten the first n% with distance_opposite sorted in descending order  
    quantile_value = df['distance_opposite'].quantile(q=(1-clipping))
    df.loc[0:(clipping * len(df)), ['distance_opposite']] = quantile_value

    return df

In [16]:
# Helper function to calculate adjsuted distance using the distance_oppsite as a penalty term

def cal_adjusted_distance(df, k=0.5):
    df['distance_adjusted'] = df.apply(lambda row: row['distance_original'] + (k * 1/row['distance_opposite']), axis='columns')
    
    return df 


In [17]:
# Helper function to calculate review level similarity scores

def cal_review_similarity_score(df):
    # find max of distance_adjusted  
    max_distance_adjusted  = df['distance_adjusted'].max()
    # normalized adjusted distance then subtract from 1 to calculate the similarity score 
    df['similarity_score'] = df['distance_adjusted'].apply(lambda x: 1 - x / max_distance_adjusted)

    return df 

In [18]:
# Helper function to calculate the product level similarity scores 

def cal_product_similarity_score(df, method='discount_reward'):

    if method == 'average':
        df_temp = df.groupby('product_id')['similarity_score'].mean()
        df_temp = df_temp.to_frame(name='product_similarity_score').reset_index()
        df = pd.merge(df, df_temp, left_on='product_id', right_on='product_id')
    else:
        df_grouped = df.groupby(by='product_id')

        for name, data in df_grouped:
            data = data.sort_values('similarity_score', ascending=False)
            scores = []
            for cnt, (index, row) in enumerate(data.iterrows()):
                discounted_score = row['similarity_score'] / pow(2, cnt)
                scores.append(discounted_score)
            df.loc[data.index, 'product_similarity_score'] = sum(scores)
        
    df = df.sort_values(by=['product_similarity_score', 'similarity_score'], ascending=[False, False])

    return df

In [19]:
def get_final_search_result(df_reconcile_result, clipping=0, weight=0, method='discount_reward'):
    """
    Method 1:   Calculate product level similarity score by Average
                clipping=0, weight=0, method='average'

    Method 2:   Calculate product level similarity score by Discount Reward
                clipping=0, weight=0, method='discount_reward' 

    Method 3:   Calcuate product level similarity score by Discount Reward with Adjustment by Opposite Query 
                clipping=0.1, weight=0.5, method='discount_reward
    """

    df_copy = df_reconcile_result.copy()

    df_copy = clip_distance_opposite(df_copy, clipping=clipping)
    df_copy = cal_adjusted_distance(df_copy, k=weight)
    df_copy = cal_review_similarity_score(df_copy)
    df_copy = cal_product_similarity_score(df_copy, method=method)

    # re-arrange columns for output
    cols = ['product_id', 'review_id', 'star_rating', 'distance_original', 'distance_opposite',
            'distance_adjusted', 'similarity_score', 'product_similarity_score', 'product_title', 'review_body']

    df_copy = df_copy[cols]

    return df_copy

### Geneate results

In [20]:
query = "Long thin cotton socks for men, need to be breathable, even feeling cool for summer time."
#query = "Wrinkle free chiffon blouse, sleek style, long sleeve, slim fit, with comfortable inside layer"

In [21]:


# 1) create the embedding for orignal query 
query_embedding = get_embedding(query)
print(f"Original Query === {query}")

# 2) search the FAISS index with orignal query embedding 
df_result_original, result_idx = search_with_original_query(df_apparel, faiss_index, query_embedding, num_of_records=100)

# 3) generate opposite query rewrite
opposite_query_rewrite = generate_opposite_query(query, prompt_rewrite)
print(f"Opposite Query (Rewrite)=== {opposite_query_rewrite}")

# 4) generate opposite query antonym
opposite_query_antonym = generate_opposite_query(query, prompt_antonym)
print(f"Opposite Query (Antonym)=== {opposite_query_antonym}")

# 5) get the embedding for the opposite query rewrite
opposite_query_embedding_rewrite = get_embedding(opposite_query_rewrite)

# 6) get the embedding for the opposite query antonym
opposite_query_embedding_antonym = get_embedding(opposite_query_antonym)

# 5) search the FAISS index with the opposite query. We need to pass the result_idx from the orignal query in order to limit the seach scope 
#df_result_opposite = search_with_opposite_query(df_apparel, faiss_index, opposite_query_embedding, result_idx, num_of_records=100)

# 6) generate the reconsile result 
#df_reconcile_result = get_reconcile_result(df_result_original, df_result_opposite)



Original Query === Long thin cotton socks for men, need to be breathable, even feeling cool for summer time.
Opposite Query (Rewrite)=== 
Short thick wool socks for men, need to be stuffy, even feeling warm for summer time.
Opposite Query (Antonym)=== Short means having little length. Thick means having a greater than usual measure across. Unbreathable means not allowing air to pass through. Hot means having or giving out a great deal of heat.


### UAMP

In [22]:
# make a func to handle the mixed- type data and return the final UMAP embeddings
def umap_embed(df, n_components=2,  n_neighbors= 15, random_state=42):
    #reducer = umap.UMAP(random_state=RAMDOM_STATE, n_components=2, metric='cosine', n_neighbors=10)
    reducer = umap.UMAP(random_state=random_state, n_components=n_components, n_neighbors=n_neighbors, metric='cosine')
    reducer.fit(df)
    umap_embedding = reducer.transform(df)
   
    return reducer, umap_embedding

In [23]:
def gen_dataframe_for_vis(umap_embedding):

    df = pd.DataFrame(umap_embedding)
    df.index.name='Review'
    df = df.reset_index()
    df.columns = [str(c) for c in df.columns]
    df = df.melt(id_vars=['Review'])
    df['variable'] = df['variable'].astype('int64')

    return df


In [24]:
df_apparel = pd.read_pickle('../resources/data/apparel_10to14_embedding.pkl')

In [25]:
df_result_original.head(3)

,product_id,review_id,star_rating,product_title,review_body,distance
64675,B00M1Y8BDG,R1D0JUBZ3UQYW6,5.0,Hanes Men's Comfortsoft Cotton Printed Lounge ...,"These are nice, light-weight lounge pants. The...",0.863644
26327,B009FG8EYE,RO1TGDIWMUXNW,5.0,Mens 100% Cotton Plain Work/Casual Socks (Pack...,These were amazing super absorbent and h elope...,0.863667
1963,B000G7WC80,R37IPWNRJ74FE3,5.0,Wigwam Men's Cool-Lite Mid Hiker Pro Quarter L...,Love these socks. Orignally got them for summ...,0.863759


In [26]:
df_result_original = df_result_original.merge(df_apparel[['review_id', 'embedding']], left_on='review_id', right_on='review_id', how='inner')

In [27]:
df_search_result_vector = pd.DataFrame(df_result_original['embedding'].values.tolist()).add_prefix('data')

In [28]:
#df_search_result_vector = pd.concat([df_search_result_vector.head(10), df_search_result_vector.tail(10)], axis=0)

In [29]:
reducer, umap_search_result = umap_embed(df_search_result_vector, n_components=10, n_neighbors=10)
df_search_result_plot = gen_dataframe_for_vis(umap_search_result[:20])

In [30]:
umap_search_result.shape

(100, 10)

In [31]:
#np.vstack((umap_search_result[:10], umap_search_result[90:])).shape

In [32]:
df_query_vector  = pd.DataFrame(query_embedding).add_prefix('data')
umap_query = reducer.transform(df_query_vector)
df_query_plot = gen_dataframe_for_vis(umap_query)

In [33]:
df_opposite_query_vector_rewrite  = pd.DataFrame(opposite_query_embedding_rewrite).add_prefix('data')
umap_opposite_query_rewrite = reducer.transform(df_opposite_query_vector_rewrite)
df_opposite_query_plot_rewrite = gen_dataframe_for_vis(umap_opposite_query_rewrite)

In [34]:
df_opposite_query_vector_antonym  = pd.DataFrame(opposite_query_embedding_antonym).add_prefix('data')
umap_opposite_query_antonym  = reducer.transform(df_opposite_query_vector_antonym)
df_opposite_query_plot_antonym  = gen_dataframe_for_vis(umap_opposite_query_antonym)

In [36]:
#df_query_plot.to_csv('../resources/data/temp/df_query_plot.csv', index=False)
#df_opposite_query_plot_rewrite.to_csv('../resources/data/temp/df_opposite_query_plot_rewrite.csv', index=False)
#df_opposite_query_plot_antonym.to_csv('../resources/data/temp/df_opposite_query_plot_antonym.csv', index=False)

In [39]:
df_query_plot = pd.read_csv('../resources/data/temp/df_query_plot.csv')
df_opposite_query_plot_rewrite = pd.read_csv('../resources/data/temp/df_opposite_query_plot_rewrite.csv')
df_opposite_query_plot_antonym = pd.read_csv('../resources/data/temp/df_opposite_query_plot_antonym.csv')

In [40]:
#color_range = ["#FFF33B", "#E93E3A"]

color_range = ["#feebe2", "#7a0177"]

query_chart = alt.Chart(data=df_query_plot).mark_rect().encode(
    x = alt.X('Review:N', title="Ori. Q."),
    y = alt.Y('variable:O', title="Component"),
    color=alt.Color('value:Q', title="Value", scale=alt.Scale(range=color_range), legend=None)
)

rewrite_chart = alt.Chart(data=df_opposite_query_plot_rewrite).mark_rect().encode(
    x = alt.X('Review:N', title="Opp. Q. Rewrite"),
    y = alt.Y('variable:O', title=""),
    color=alt.Color('value:Q', title="Value", scale=alt.Scale(range=color_range), legend=None)
)

antonym_chart = alt.Chart(data=df_opposite_query_plot_antonym).mark_rect().encode(
    x = alt.X('Review:N', title="Opp. Q. Antonym"),
    y = alt.Y('variable:O', title=""),
    color=alt.Color('value:Q', title="Value", scale=alt.Scale(range=color_range), legend=None)
)

search_result_chart = alt.Chart(data=df_search_result_plot).mark_rect().encode(
    x = alt.X('Review:N', title="Search Result (0-19)"),
    y = alt.Y('variable:O', title=""),
    color=alt.Color('value:Q', title="Value", scale=alt.Scale(range=color_range))
)

query_chart | rewrite_chart | antonym_chart | search_result_chart

alt.HConcatChart(...)